# Practice assignment: Advanced ensembling techniques


In this programming assignment, you are going to work with a dataset based on the following data:

https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity

_Citation:_

* _K. Fernandes, P. Vinagre and P. Cortez. A Proactive Intelligent Decision Support System for Predicting the Popularity of Online News. Proceedings of the 17th EPIA 2015 - Portuguese Conference on Artificial Intelligence, September, Coimbra, Portugal._

The dataset contains the information about the internet news articles. In this assignment, you are going to predict a number of shares of the news article (target column: `shares`). The information about the features is available through the link above. You are going to construct several machine learning algorithms (XGBoost, LightGBM, CatBoost and Lasso) and blend them into the final ensemble.

In [2]:
# !pip install lightgbm
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [3]:
df = pd.read_csv('data.csv')

## 1

**q1:** How many missing values are there in the data? Provide the number of cells in the dataframe that contain NaNs.

In [4]:
# your code here
df.isna().sum()

n_tokens_title                   0
n_tokens_content                 0
n_unique_tokens                  0
n_non_stop_words                 0
n_non_stop_unique_tokens         0
num_hrefs                        0
num_self_hrefs                   0
num_imgs                         0
num_videos                       0
average_token_length             0
num_keywords                     0
data_channel_is_lifestyle        0
data_channel_is_entertainment    0
data_channel_is_bus              0
data_channel_is_socmed           0
data_channel_is_tech             0
data_channel_is_world            0
kw_min_min                       0
kw_max_min                       0
kw_avg_min                       0
kw_min_max                       0
kw_max_max                       0
kw_avg_max                       0
kw_min_avg                       0
kw_max_avg                       0
kw_avg_avg                       0
self_reference_min_shares        0
self_reference_max_shares        0
self_reference_avg_s

## 2

**q2:** What is the maximum number of shares among all the news articles presented in the data?

In [5]:
# your code here
df.shares.max()

843300

## 3

**q3:** What is the median number of shares for the articles published on Monday?

In [6]:
df[df.weekday_is_monday == 1].shares.median()

1400.0

## 4

First, we separate the target from the dataframe with features (`df` -> `X`, `y`).

Next, let's split the data into train/val/test sets in the ratio 60:20:20. The idea is that we will use train set to train our models, val set to validate them and test set to calculate the final error of the blend. So, test set will be a completely unseen data.

To do this, use a regular `train_test_split` from `sklearn` to split `X` and `y` into train and val/test parts in the ratio 60:40. Then use `train_test_split` again, but to split the obtain val/test part into validation and test in the ratio 50:50. In each `train_test_split` application, use `random_state=13` and other default parameter values.

In the end, you should obtain `X_train`, `X_val`, `X_test` with the following shapes, respectively: (23786, 58), (7929, 58), (7929, 58). The same logic is with `y_train`, `y_val`, `y_test`.

**q4:** What is the mean value of target in the test part (`X_test`)? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [7]:
X = df.drop('shares', axis=1)
y = df['shares']


In [8]:
# your code here
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=13)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=13)

X_train.shape, X_val.shape, X_test.shape

((23786, 58), (7929, 58), (7929, 58))

In [9]:
round(y_test.mean(), 5)

3349.74057

## 5

Now let's train our first model - XGBoost. A link to the documentation: https://xgboost.readthedocs.io/en/latest/

We will use Scikit-Learn Wrapper interface for XGBoost (and the same logic applies to the following LightGBM and CatBoost models). Here, we work on the regression task - hence we will use `XGBRegressor`. Read about the parameters of `XGBRegressor`: https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor

The main list of XGBoost parameters: https://xgboost.readthedocs.io/en/latest/parameter.html# Look through this list so that you understand which parameters are presented in the library.

Take `XGBRegressor` with MSE objective (`objective='reg:squarederror'`), 200 trees (`n_estimators=200`), `learning_rate=0.01`, `max_depth=5`, `random_state=13` and all other default parameter values. Train it on the train set (`fit` function). 

**q5:** Calculate Root Mean Squared Error (RMSE) on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [10]:
# your code here
xgbr = XGBRegressor(objective='reg:squarederror', n_estimators=200, learning_rate=0.01, max_depth=5, random_state=13) 
xgbr.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=5, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=13, ...)

In [11]:
y_val_pred = xgbr.predict(X_val)

In [12]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(y_val, y_val_pred)**0.5

In [13]:
round(rmse, 5)

10329.20768

## 6

In the task 5, we have decided to build 200 trees in our model. However, it is hard to understand whether it is a good decision - maybe it is too much? Maybe 150 is a better number? Or 100? Or 50 is enough?

During the training process, it is possible to stop constructing the ensemble if we see that the validation error does not decrease anymore. Using the same XGBoost model, call `fit` function (to train it) with `eval_set=[(X_val, y_val)]` (to evaluate the boosting model after building a new tree) and `early_stopping_rounds=50` (and other default parameter values). This `early_stopping_rounds` says that if the validation metric does not increase on 50 consequent iterations, the training stops.

**q6:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [14]:
# your code here

xgbr.fit(X_train, y_train, 
            eval_set=[(X_val, y_val)], 
            early_stopping_rounds=50) 

[0]	validation_0-rmse:9132.57896
[1]	validation_0-rmse:9118.46796
[2]	validation_0-rmse:9104.95068
[3]	validation_0-rmse:9091.79293
[4]	validation_0-rmse:9079.19770


/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[5]	validation_0-rmse:9067.49453
[6]	validation_0-rmse:9056.05565
[7]	validation_0-rmse:9045.18382
[8]	validation_0-rmse:9035.23396
[9]	validation_0-rmse:9025.48494
[10]	validation_0-rmse:9016.13997
[11]	validation_0-rmse:9007.63286
[12]	validation_0-rmse:8999.20179
[13]	validation_0-rmse:8991.06644
[14]	validation_0-rmse:8983.98021
[15]	validation_0-rmse:8976.73861
[16]	validation_0-rmse:8970.26101
[17]	validation_0-rmse:8963.87773
[18]	validation_0-rmse:8958.61072
[19]	validation_0-rmse:8953.27067
[20]	validation_0-rmse:8948.50669
[21]	validation_0-rmse:8944.50485
[22]	validation_0-rmse:8938.09517
[23]	validation_0-rmse:8934.24954
[24]	validation_0-rmse:8931.12920
[25]	validation_0-rmse:8925.65214
[26]	validation_0-rmse:8923.34015
[27]	validation_0-rmse:8918.59897
[28]	validation_0-rmse:8916.47308
[29]	validation_0-rmse:8912.38908
[30]	validation_0-rmse:8908.04187
[31]	validation_0-rmse:8907.26784
[32]	validation_0-rmse:8906.41027
[33]	validation_0-rmse:8903.25394
[34]	validation_0-r

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=5, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=13, ...)

In [15]:
y_pred = xgbr.predict(X_val)
rmse = mean_squared_error(y_val, y_pred)**0.5
round(rmse, 5)

8890.23464

## 7

Notes on parameter tuning: https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html

Here, we tuned some parameters of the algorithm. Take `XGBRegressor` with the following parameters:

* `objective='reg:squarederror'`
* `n_estimators=5000`
* `learning_rate=0.001`
* `max_depth=4`
* `gamma=1`
* `subsample=0.5`
* `random_state=13`
* all other default parameter values

Train it in the same manner, as in the task 6, but with `early_stopping_rounds=500`. 

**q7:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Notice the speed of the algorithm.

In [16]:
# your code here
model_2 = XGBRegressor(objective='reg:squarederror', n_estimators=5000, learning_rate=0.001, max_depth=4,
                      gamma=1, subsample=0.5, random_state=13) 
model_2.fit(X_train, y_train, 
            eval_set=[(X_val, y_val)], 
            early_stopping_rounds=500) 

y_pred = model_2.predict(X_val)
rmse = mean_squared_error(y_val, y_pred)**0.5
round(rmse, 5)

[0]	validation_0-rmse:9145.78842
[1]	validation_0-rmse:9144.42063
[2]	validation_0-rmse:9143.00983
[3]	validation_0-rmse:9141.59376
[4]	validation_0-rmse:9140.17952
[5]	validation_0-rmse:9138.77550
[6]	validation_0-rmse:9137.15972


/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[7]	validation_0-rmse:9135.65232
[8]	validation_0-rmse:9134.11349
[9]	validation_0-rmse:9132.71948
[10]	validation_0-rmse:9131.40694
[11]	validation_0-rmse:9129.92722
[12]	validation_0-rmse:9128.53498
[13]	validation_0-rmse:9127.15092
[14]	validation_0-rmse:9125.81464
[15]	validation_0-rmse:9124.47845
[16]	validation_0-rmse:9122.91544
[17]	validation_0-rmse:9121.50081
[18]	validation_0-rmse:9120.15113
[19]	validation_0-rmse:9118.70679
[20]	validation_0-rmse:9117.30311
[21]	validation_0-rmse:9115.88248
[22]	validation_0-rmse:9114.39763
[23]	validation_0-rmse:9112.98066
[24]	validation_0-rmse:9111.45363
[25]	validation_0-rmse:9110.14621
[26]	validation_0-rmse:9108.71929
[27]	validation_0-rmse:9107.29806
[28]	validation_0-rmse:9105.95208
[29]	validation_0-rmse:9104.58900
[30]	validation_0-rmse:9103.23103
[31]	validation_0-rmse:9101.82554
[32]	validation_0-rmse:9100.43117
[33]	validation_0-rmse:9099.14855
[34]	validation_0-rmse:9097.74098
[35]	validation_0-rmse:9096.33481
[36]	validation_0

[244]	validation_0-rmse:8874.42119
[245]	validation_0-rmse:8873.50515
[246]	validation_0-rmse:8872.58010
[247]	validation_0-rmse:8871.71013
[248]	validation_0-rmse:8871.08177
[249]	validation_0-rmse:8870.36210
[250]	validation_0-rmse:8869.47498
[251]	validation_0-rmse:8868.80058
[252]	validation_0-rmse:8867.94602
[253]	validation_0-rmse:8867.27876
[254]	validation_0-rmse:8866.61451
[255]	validation_0-rmse:8866.09402
[256]	validation_0-rmse:8865.22128
[257]	validation_0-rmse:8864.37969
[258]	validation_0-rmse:8863.51544
[259]	validation_0-rmse:8862.67123
[260]	validation_0-rmse:8861.73516
[261]	validation_0-rmse:8861.06955
[262]	validation_0-rmse:8860.45578
[263]	validation_0-rmse:8859.58217
[264]	validation_0-rmse:8858.78559
[265]	validation_0-rmse:8858.09208
[266]	validation_0-rmse:8857.15210
[267]	validation_0-rmse:8856.48894
[268]	validation_0-rmse:8855.86106
[269]	validation_0-rmse:8855.17199
[270]	validation_0-rmse:8854.30886
[271]	validation_0-rmse:8853.62754
[272]	validation_0-r

[479]	validation_0-rmse:8742.95156
[480]	validation_0-rmse:8742.78938
[481]	validation_0-rmse:8742.70883
[482]	validation_0-rmse:8742.16194
[483]	validation_0-rmse:8741.57363
[484]	validation_0-rmse:8741.40829
[485]	validation_0-rmse:8741.06796
[486]	validation_0-rmse:8740.76855
[487]	validation_0-rmse:8740.49162
[488]	validation_0-rmse:8740.18496
[489]	validation_0-rmse:8740.03953
[490]	validation_0-rmse:8739.78528
[491]	validation_0-rmse:8739.34260
[492]	validation_0-rmse:8739.29895
[493]	validation_0-rmse:8739.09457
[494]	validation_0-rmse:8738.56560
[495]	validation_0-rmse:8738.39737
[496]	validation_0-rmse:8738.15312
[497]	validation_0-rmse:8737.96486
[498]	validation_0-rmse:8737.49652
[499]	validation_0-rmse:8737.33942
[500]	validation_0-rmse:8736.92123
[501]	validation_0-rmse:8736.71213
[502]	validation_0-rmse:8736.59608
[503]	validation_0-rmse:8736.13088
[504]	validation_0-rmse:8735.55918
[505]	validation_0-rmse:8735.43507
[506]	validation_0-rmse:8735.27973
[507]	validation_0-r

[714]	validation_0-rmse:8696.51750
[715]	validation_0-rmse:8696.70406
[716]	validation_0-rmse:8696.39230
[717]	validation_0-rmse:8696.07345
[718]	validation_0-rmse:8695.96114
[719]	validation_0-rmse:8696.04177
[720]	validation_0-rmse:8696.34223
[721]	validation_0-rmse:8696.39895
[722]	validation_0-rmse:8696.07263
[723]	validation_0-rmse:8696.13510
[724]	validation_0-rmse:8695.91593
[725]	validation_0-rmse:8695.59127
[726]	validation_0-rmse:8695.88948
[727]	validation_0-rmse:8695.94765
[728]	validation_0-rmse:8695.65654
[729]	validation_0-rmse:8695.95527
[730]	validation_0-rmse:8696.06360
[731]	validation_0-rmse:8695.89946
[732]	validation_0-rmse:8695.57539
[733]	validation_0-rmse:8695.69627
[734]	validation_0-rmse:8695.90891
[735]	validation_0-rmse:8695.59775
[736]	validation_0-rmse:8695.25503
[737]	validation_0-rmse:8695.10824
[738]	validation_0-rmse:8695.21606
[739]	validation_0-rmse:8695.46766
[740]	validation_0-rmse:8695.72335
[741]	validation_0-rmse:8695.91663
[742]	validation_0-r

[949]	validation_0-rmse:8699.66171
[950]	validation_0-rmse:8699.88244
[951]	validation_0-rmse:8699.66919
[952]	validation_0-rmse:8699.44052
[953]	validation_0-rmse:8699.71014
[954]	validation_0-rmse:8699.57341
[955]	validation_0-rmse:8699.82729
[956]	validation_0-rmse:8700.15181
[957]	validation_0-rmse:8700.15309
[958]	validation_0-rmse:8699.98845
[959]	validation_0-rmse:8699.73887
[960]	validation_0-rmse:8699.62170
[961]	validation_0-rmse:8700.24433
[962]	validation_0-rmse:8700.06568
[963]	validation_0-rmse:8700.47493
[964]	validation_0-rmse:8700.56701
[965]	validation_0-rmse:8700.50641
[966]	validation_0-rmse:8700.32579
[967]	validation_0-rmse:8700.33583
[968]	validation_0-rmse:8700.60496
[969]	validation_0-rmse:8701.11288
[970]	validation_0-rmse:8700.89323
[971]	validation_0-rmse:8700.67038
[972]	validation_0-rmse:8700.54519
[973]	validation_0-rmse:8700.95332
[974]	validation_0-rmse:8701.22283
[975]	validation_0-rmse:8701.64937
[976]	validation_0-rmse:8701.45940
[977]	validation_0-r

[1178]	validation_0-rmse:8750.05147
[1179]	validation_0-rmse:8750.43135
[1180]	validation_0-rmse:8750.52910
[1181]	validation_0-rmse:8751.00118
[1182]	validation_0-rmse:8750.91763
[1183]	validation_0-rmse:8750.78772
[1184]	validation_0-rmse:8750.66262
[1185]	validation_0-rmse:8750.54528
[1186]	validation_0-rmse:8751.28853
[1187]	validation_0-rmse:8751.82557
[1188]	validation_0-rmse:8752.23776
[1189]	validation_0-rmse:8751.99008
[1190]	validation_0-rmse:8752.03957
[1191]	validation_0-rmse:8752.19066
[1192]	validation_0-rmse:8752.23019
[1193]	validation_0-rmse:8752.18772
[1194]	validation_0-rmse:8752.04799
[1195]	validation_0-rmse:8751.93196
[1196]	validation_0-rmse:8752.34399
[1197]	validation_0-rmse:8753.20809
[1198]	validation_0-rmse:8753.10948
[1199]	validation_0-rmse:8753.65014
[1200]	validation_0-rmse:8753.56454
[1201]	validation_0-rmse:8753.93350
[1202]	validation_0-rmse:8754.36318
[1203]	validation_0-rmse:8754.65477
[1204]	validation_0-rmse:8755.48216
[1205]	validation_0-rmse:875

8692.77417

## 8

Calculate feature importances according to the model, trained in the task 7. 

**q8:** What is the name of the most important feature? Provide it as the answer. Do you understand why it might be important for the model?

Notice that by default, `XGBRegressor` calculates feature importance considering gain (`importance_type` parameter).

In [17]:
# your code here
from matplotlib import pyplot as plt
from xgboost import plot_importance

imp_dict = {}
importances = model_2.feature_importances_
for i,v in enumerate(importances):
    imp_dict[i]= v

In [18]:
max(imp_dict, key=imp_dict.get)

13

In [19]:
df.columns[13]


'data_channel_is_bus'

## 9

Let's move to LightGBM. We will work with `LGBMRegressor`.

LGBMRegressor parameters: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html#lightgbm.LGBMRegressor

The main list of LightGBM parameters: https://lightgbm.readthedocs.io/en/latest/Parameters.html Look through this list so that you understand which parameters are presented in the library.

Take `LGBMRegressor` with the following parameters, similar to the previous `XGBoost` model:

* `objective='regression'`
* `n_estimators=200`
* `learning_rate=0.01`
* `max_depth=5`
* `random_state=13`
* other default parameter values

Train it on the training data with `eval_set=[(X_val, y_val)]`, `eval_metric='rmse'`, `early_stopping_rounds=50` and all other default parameter values. 

**q9:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Notice the speed of the algorithm and compare it to the speed of XGBoost model.

In [20]:
# your code here

lgbm_model = LGBMRegressor(objective='regression', n_estimators=200, learning_rate=0.01,
                          max_depth=5, random_state=13)



In [1]:
lgbm_model.fit(X_train, y_train) 



NameError: name 'lgbm_model' is not defined

## 10

Notes on parameter tuning: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

Here, we tuned some parameters of the algorithm. Take `LGBMRegressor` with the following parameters:

* `objective='regression'`
* `n_estimators=5000`
* `learning_rate=0.001`
* `max_depth=3`
* `lambda_l2=1.0`
* `boosting_type='goss'`
* `random_state=13`
* all other default parameter values

Train it in the same manner, as in the task 9, but with `early_stopping_rounds=500`. 

**q10:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [ ]:
# your code here

## 11

Calculate feature importances according to the model, trained in the task 10. 

**q11:** What is the name of the most important feature? Provide it as the answer. 

Do you understand why it might be important for the model?

Notice that by default, `LGBMRegressor` calculates feature importance considering number of times the feature is used in the model (`importance_type` parameter).

In [ ]:
# your code here

## 12

Since some features are not important for the model, we can drop them in order to try to construct a better model which does not consider them at all.

Obtain new train and validation sets without the features with LightGBM importance less than 10 (the importances were computed in the task 11). Train the same model as in the task 10 on the new train set in the same manner. 

**q12:** Calculate RMSE on the new validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Notice that the new versions of train and validation sets are used only in this task and in blending.

In [ ]:
# your code here

## 13

Let's move to CatBoost. We will work with `CatBoostRegressor`.

Info about `CatBoostRegressor`: https://catboost.ai/docs/concepts/python-reference_catboostregressor.html

CatBoost parameters: https://catboost.ai/docs/concepts/python-reference_parameters-list.html Look through this list so that you understand which parameters are presented in the library.

Take `CatBoostRegressor` with the following parameters, similar to the previous models:

* `loss_function='RMSE'`
* `iterations=200`
* `learning_rate=0.01`
* `max_depth=5`
* `random_state=13`
* other default parameter values

Train it on the training data with `eval_set=[(X_val, y_val)]`, `early_stopping_rounds=50` and all other default parameter values. 

**q13:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Notice the speed of the algorithm and compare it to the speed of XGBoost and LightGBM models.

In [ ]:
# your code here

## 14

Notes on parameter tuning: https://catboost.ai/docs/concepts/parameter-tuning.html

Here, we tuned some parameters of the algorithm. Take `CatBoostRegressor` with the following parameters:

* `loss_function='RMSE'`
* `n_estimators=5000`
* `learning_rate=0.001`
* `max_depth=9`
* `random_state=13`
* all other default parameter values

Train it in the same manner, as in the task 13, but with `early_stopping_rounds=500`. 

**q14:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [ ]:
# your code here

## 15

Calculate feature importances according to the model, trained in the task 14. 

**q15:** What is the name of the most important feature? Provide it as the answer. 

Do you understand why it might be important for the model?

Notice that in case of regression, `CatBoostRegressor` calculates feature importance considering PredictionValuesChange: https://catboost.ai/docs/concepts/fstr.html#fstr__regular-feature-importance

In [ ]:
# your code here

## 16

Finally, take a `Lasso` model from `sklearn` with `alpha=10.0`, `random_state=13` and all other default parameter values. Train it on the train set. 

**q16:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [ ]:
# your code here

## 17

Compare the results on the validation set of the trained models:

* XGBoost (task 7)
* LightGBM (task 12)
* CatBoost (task 14)
* Lasso (task 16)

**q17:** Which model has the best RMSE value on the validation set? For the answer, provide the following:

* 1 (if XGBoost was the best)
* 2 (if LightGBM was the best)
* 3 (if CatBoost was the best)
* 4 (if Lasso was the best)

## 18

Finally, let's move to blending the models that we obtained. First, calculate the predictions for the trained models on the validation set. Remember that LightGBM model used slightly different set of columns in the data.

After getting the predictions for the validation set, concatenate them into a single dataframe `X_val_blend`. The dataframe should look like this:

||xgb|lgb|cb|lasso|
|-|-|-|-|-|
|0|2298.947754|3728.088336|3680.924182|4270.039931|
|1|3208.189209|5243.744431|4487.549790|6755.853939|
|...|...|...|...|...|

Here, `xgb` column represents XGBoost predictions, `lgb` - LightGBM predictions, `cb` - CatBoost predictions, `lasso` - lasso predictions.

**q18:** For the answer, calculate the mean value of all model predictions in the last row of this column (`X_val_blend.iloc[-1]`). Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [ ]:
# your code here

## 19

Obtain a matrix of pairwise Pearson Correlation Coefficient (PCC) values for the column of the dataframe `X_val_blend`. Find a pair of model predictions with the highest PCC value (don't consider 1.0 values of correlations with themselves). 

**q19:** What is this value equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [ ]:
# your code here

## 20

Blend models into the ensemble with the weights 0.25, 0.25, 0.25 and 0.25 (just mean value of the predictions). 

**q20:** Calculate RMSE of such ensemble on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Compare it with RMSE of each model and think whether this is a good ensemble.

In [ ]:
# your code here

## 21

Tune the weights of the ensemble. Run each model weight through `np.linspace(0, 1, 101)`, so that all possible values of each weight will be [0.0, 0.01, 0.02, ..., 0.99, 1.0]. Skip each combinations of weights, if their sum is not equal to 1.0. If the sum of the weights in the combination is equal to 1.0, though, get ensemble prediction on the validation set using these weights and calculate RMSE value.

In the end, select a combination of weights with the best RMSE value - these are the best weights for the ensemble. 

**q21:** What is their corresponding RMSE value equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Compare RMSE value of the ensemble with RMSE values of the models in it. Is the ensemble better?

_Hint. You probably want to save RMSE with the corresponding weights for each valid combination into some array. Also this weight tuning might be implemented as quadriple nested loop, or you may think about other ways of implementing it. You can track tuning progress using `tqdm` module._

In [ ]:
# your code here

## 22

Using the best weights obtained in the task 21, run the best ensemble on the test set. To do this, obtain model predictions on the test set (you can write them to the similar table to the one for the validation set in the task 18). Remember that LightGBM model uses slightly different set of columns.

**q22:** Calculate RMSE of the final ensemble on the test set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [ ]:
# your code here